In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

In [12]:
program = "maximal_matching"

merge_files = [
    "w_exp_maximal_matching__graph_7__0__10000__2__0__duong",
    "w_exp_maximal_matching__graph_7__0__10000__2__4__duong",
    "w_exp_maximal_matching__graph_7__0__10000__2__9__duong",
]

selected_nodes = [
    i.split("__")[5] for i in merge_files
]
selected_nodes

['0', '4', '9']

In [7]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv") for f in merge_files]
dfs

[      Steps  Count    exp_fit
 0        12     10  72.342522
 1        12     33  71.827245
 2        13     10  71.315629
 3        14     50  70.807646
 4        15     15  70.303271
 ...     ...    ...        ...
 1435   1435      0  -0.200354
 1436   1436      0  -0.200372
 1437   1437      0  -0.200391
 1438   1438      0  -0.200409
 1439   1439      1  -0.200427
 
 [1440 rows x 3 columns],
       Steps  Count    exp_fit
 0        10      7  61.743761
 1        10     26  61.369228
 2        11      7  60.996961
 3        12     44  60.626948
 4        13      6  60.259174
 ...     ...    ...        ...
 1751   1751      0  -0.141952
 1752   1752      0  -0.141962
 1753   1753      0  -0.141971
 1754   1754      0  -0.141979
 1755   1755      1  -0.141988
 
 [1756 rows x 3 columns],
       Steps  Count    exp_fit
 0         5      6  57.611150
 1         5      3  57.288068
 2         6      9  56.966792
 3         7      2  56.647312
 4         8     20  56.329620
 ...     ...  

In [10]:
dfs[2]

,Steps,Count,exp_fit
0,5,6,57.611150
1,5,3,57.288068
2,6,9,56.966792
3,7,2,56.647312
4,8,20,56.329620
...,...,...,...
2073,2073,0,-0.159263
2074,2074,0,-0.159266
2075,2075,0,-0.159268
2076,2076,0,-0.159271


In [17]:
df_merged = pd.DataFrame(columns=["Steps"])
# for i in range(1, len(dfs)):
#     df_merged = pd.merge(df_merged, dfs[i], on=["Steps"], suffixes=(i-1, i))
# df_merged
df_merged = pd.merge(dfs[0], dfs[1], on=["Steps"], suffixes=(0, 1))
for i in range(2, len(dfs)):
    df_merged = pd.merge(df_merged, dfs[i], suffixes=(i-1, i))

df_merged

,Steps,Count0,exp_fit0,Count1,exp_fit1,Count,exp_fit
0,12,10,72.342522,44,60.626948,25,55.076518
1,12,33,71.827245,44,60.626948,25,55.076518
2,13,10,71.315629,6,60.259174,7,54.767610
3,14,50,70.807646,36,59.893625,43,54.460430
4,15,15,70.303271,11,59.530289,12,54.154967
...,...,...,...,...,...,...,...
1446,1435,0,-0.200354,0,-0.133190,0,-0.141198
1447,1436,0,-0.200372,0,-0.133252,0,-0.141302
1448,1437,0,-0.200391,0,-0.133313,0,-0.141405
1449,1438,0,-0.200409,0,-0.133375,0,-0.141508
